In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI as Chat
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate

load_dotenv()

True

In [2]:
openai_api_key = os.getenv("OPENAI_API_KEY")
model = os.getenv("OPENAI_MODEL", "gpt-4o")
temperature = float(os.getenv("OPENAI_TEMPERATURE", 0))

NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
llm = Chat(
    openai_api_key=openai_api_key,
    model=model,
    temperature=temperature
)

In [4]:
graph = Neo4jGraph(
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

In [5]:
# Define the Cypher generation template
CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
Convert the user's question based on the schema.

Schema: {schema}
Question: {question}
"""

In [6]:
# Create a prompt template for Cypher generation
cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

print(cypher_generation_prompt)

input_variables=['question', 'schema'] input_types={} partial_variables={} template="\nYou are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.\nConvert the user's question based on the schema.\n\nSchema: {schema}\nQuestion: {question}\n"


In [7]:
# Initialize the GraphCypherQAChain with the language model, graph, and prompt template
cypher_chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    cypher_prompt=cypher_generation_prompt,
    verbose=True,
    allow_dangerous_requests=True
)

print(cypher_chain)

verbose=True graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x111f57110> cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template="\nYou are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.\nConvert the user's question based on the schema.\n\nSchema: {schema}\nQuestion: {question}\n"), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x111fa47d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x111fa6420>, root_client=<openai.OpenAI object at 0x110db01d0>, root_async_client=<openai.AsyncOpenAI object at 0x111fa4830>, model_name='gpt-4o', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********')), output_parser=StrOutputParser(), llm_kwargs={}) qa_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['con

In [27]:
# Invoke the chain with a sample query and print the result
response = cypher_chain.invoke(
    {"query": "Give info about each node in the graph"},
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (n)
RETURN labels(n) AS NodeType, properties(n) AS NodeProperties

Full Context:
[{'NodeType': ['Session'], 'NodeProperties': {'id': 'aee31963-d525-4cfb-9b71-f9dae62a1ccf'}}, {'NodeType': ['Message'], 'NodeProperties': {'content': 'hi', 'type': 'human'}}, {'NodeType': ['Message'], 'NodeProperties': {'content': "Hey dude! What's up? You ready to catch some gnarly waves today? 🌊🤙", 'type': 'ai'}}, {'NodeType': ['Message'], 'NodeProperties': {'content': 'how is watergate?', 'type': 'human'}}, {'NodeType': ['Message'], 'NodeProperties': {'content': "Ah, Watergate Bay, bro! It's looking like a chill sesh today with 3ft waves and some onshore winds. Not the most epic conditions, but you can still have a fun time shredding those smaller sets. Just keep it mellow and enjoy the ride, man! 🏄\u200d♂️🌊", 'type': 'ai'}}, {'NodeType': ['Message'], 'NodeProperties': {'content': '', 'type': 'human'}}, {'NodeType': ['Message'],

In [28]:
print(response["result"])

The graph contains the following nodes:

1. A Session node with the ID: aee31963-d525-4cfb-9b71-f9dae62a1ccf.
2. A Message node with the content: "hi" and type: human.
3. A Message node with the content: "Hey dude! What's up? You ready to catch some gnarly waves today? 🌊🤙" and type: ai.
4. A Message node with the content: "how is watergate?" and type: human.
5. A Message node with the content: "Ah, Watergate Bay, bro! It's looking like a chill sesh today with 3ft waves and some onshore winds. Not the most epic conditions, but you can still have a fun time shredding those smaller sets. Just keep it mellow and enjoy the ride, man! 🏄‍♂️🌊" and type: ai.
6. A Message node with an empty content and type: human.
7. A Message node with the content: "Hey, you still there, dude? If you're thinking about hitting up another spot or need some tips, just let me know! 🤙🌊" and type: ai.
8. A Message node with an empty content and type: human.
9. A Message node with the content: "No worries, bro! Just 